In [1]:
data_var = '2023-12-11'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7829,2023-12-11,Eua Nba,21:00,Charlotte Hornets,Miami Heat,2.44,1.65,221.5,1.80,2.08,3.5,1.97,1.97,AgI0Cqin,0.409836,0.606061,0.555556,0.480769,0.015897,0.6,0.2,NaN,NaN,398.016,134.483744,0.337885,236.220,39.021317,0.165191,287.772,231.806,321.30,252.00,0.0,0.0,0.0,0.0,0.273161,0.102057,0.000000,-0.69,-0.138,-10.434783,0.407629,0.3,-0.107629,-0.90,-0.180,-3.611111,0.468858,0.5,0.031142
7830,2023-12-11,Eua Nba,21:00,Detroit Pistons,Indiana Pacers,3.23,1.42,245.5,1.83,2.05,6.5,2.05,1.89,4GH4B36h,0.309598,0.704225,0.546448,0.487805,0.013823,0.8,1.0,NaN,NaN,475.638,240.745172,0.506152,284.766,122.037543,0.428554,303.146,425.618,205.02,368.64,0.0,0.0,0.0,0.0,0.550479,0.080187,0.057430,-5.00,-1.000,-2.230000,0.359867,0.1,-0.259867,2.80,0.560,0.750000,0.263617,0.5,0.236383
7831,2023-12-11,Eua Nba,21:00,Orlando Magic,Cleveland Cavaliers,1.78,2.20,224.5,1.86,2.07,-3.5,2.09,1.85,IJL8ANLb,0.561798,0.454545,0.537634,0.483092,0.016343,0.6,0.4,NaN,NaN,200.508,64.361091,0.320990,172.750,42.336239,0.245072,193.048,228.598,146.37,218.67,0.0,0.0,0.0,0.0,0.149239,0.075569,0.086145,2.77,0.554,1.407942,0.596852,0.8,0.203148,2.91,0.582,2.061856,0.534639,0.7,0.165361
7832,2023-12-11,Eua Nba,21:00,Philadelphia 76ers,Washington Wizards,1.15,7.04,241.5,1.89,2.04,-13.5,2.06,1.87,0rBD9sy5,0.869565,0.142045,0.529101,0.490196,0.011611,0.8,0.8,NaN,NaN,228.690,90.335935,0.395015,527.676,169.962663,0.322097,186.472,598.944,166.25,408.37,0.0,0.0,0.0,0.0,1.017060,0.053978,0.068372,-2.17,-0.434,-0.345622,0.655883,0.7,0.044117,-2.57,-0.514,-11.750973,0.270691,0.2,-0.070691
7833,2023-12-11,Eua Nba,21:30,Atlanta Hawks,Denver Nuggets,2.25,1.75,238.5,1.89,2.04,1.5,2.09,1.85,vZ9H81jB,0.444444,0.571429,0.529101,0.490196,0.015873,0.6,0.6,NaN,NaN,295.042,113.386683,0.384307,253.824,162.036737,0.638382,205.060,261.106,177.41,259.74,0.0,0.0,0.0,0.0,0.176777,0.053978,0.086145,-3.43,-0.686,-1.822157,0.526265,0.4,-0.126265,-0.30,-0.060,-12.500000,0.554765,0.4,-0.154765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7895,2023-12-11,Israel Wbl Feminina,14:00,Maccabi Ashdod F,Maccabi Haifa F,1.20,4.17,129.5,1.85,1.85,-12.5,2.07,1.67,pE7hTXdB,0.833333,0.239808,0.540541,0.540541,0.073141,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.782163,0.000000,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7896,2023-12-11,Israel Wbl Feminina,14:00,Maccabi Urban F,Ramat Hasharon F,1.25,3.69,130.5,1.81,1.89,-11.5,2.07,1.67,EmCdSiCH,0.800000,0.271003,0.552486,0.529101,0.071003,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.698518,0.030578,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7897,2023-12-11,Rússia Superliga,11:00,Chelyabinsk,University-Ugra,2.53,1.48,147.5,1.85,1.85,2.5,2.07,1.67,048ENAsg,0.395257,0.675676,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,146.290,45.928349,0.313954,NaN,NaN,NaN,0.000,0.000,88.80,134.40,0.0,0.0,0.0,0.0,0.370305,0.000000,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7898,2023-12-11,Rússia Superliga,12:00,Zenit Petersburg 2,Novosibirsk,2.07,1.70,149.5,1.85,1.85,1.5,1.92,1.79,jg1RK8RB,0.483092,0.588235,0.540541,0.540541,0.071327,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Charlotte Hornets,Miami Heat,221.5,1.80,1.0000,Over
1,21:00,Eua Nba,Detroit Pistons,Indiana Pacers,245.5,1.83,1.0000,Over
2,21:00,Eua Nba,Orlando Magic,Cleveland Cavaliers,224.5,1.86,1.0000,Over
3,21:00,Eua Nba,Philadelphia 76ers,Washington Wizards,241.5,1.89,1.0000,Over
4,21:30,Eua Nba,Atlanta Hawks,Denver Nuggets,238.5,1.89,1.0000,Over
5,21:30,Eua Nba,New York Knicks,Toronto Raptors,218.5,1.80,0.9999,Over
6,22:00,Eua Nba,Houston Rockets,San Antonio Spurs,223.5,1.86,1.0000,Over
7,22:00,Eua Nba,Memphis Grizzlies,Dallas Mavericks,225.5,1.82,1.0000,Over
8,22:00,Eua Nba,Milwaukee Bucks,Chicago Bulls,231.5,1.82,1.0000,Over
9,22:00,Eua Nba,New Orleans Pelicans,Minnesota Timberwolves,221.5,1.86,1.0000,Over
